In [11]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
min_loss = 0
max_profit = 0
commission = 5


In [12]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

In [13]:
symbols = """
select distinct(name) from stockdataweekly order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [14]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [15]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = None
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        

sql1= """
select distinct(name) from stockdataweekly order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)

<h1> Updating resources

In [16]:
from tqdm import tqdm

pbar = tqdm(total= len(symbol_list))

for symbol in symbol_list:
    symbol_data = """
    select * from stockdataweekly where name = '%s'
    """ % symbol
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)
    pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████▉| 3869/3870 [02:18<00:00, 29.48it/s]

In [17]:
sql_dates = """
select distinct(date) from stockdataweekly order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [18]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global min_loss
    global max_profit
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdataweekly where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:                    
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        buy_stocks(current_price, current_symbol, date)   

                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                win += 1
                                current_symbol.flag_for_wins = True
                        
                        
                    if current_symbol.stop_order > current_price:                        
                        sell_stocks(current_symbol, current_price, date)
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [19]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global risk
    global commission
    
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))
              
    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)
    else:
        pass

def sell_stocks(current_symbol, current_price, date):
    global position
    global remaining_cash
    global profit
    global commission
    global min_loss
    global max_profit
    global win
    global loss
    
    current_symbol.stop_order = None
    print("Selling ",position[current_symbol.symbol], "stocks of", current_symbol.symbol, " at price", current_price, "on date", date)
    sold = current_price * position[current_symbol.symbol] - commission
    profit_today = sold - current_symbol.bought * position[current_symbol.symbol]
    if profit_today > 0:
        if current_symbol.flag_for_wins == False:
            win += 1
        current_symbol.flag_for_wins = False

    else:
        loss += 1
        current_symbol.flag_for_wins = False

    profit += profit_today

    if profit < min_loss:
        min_loss = profit
    if profit > max_profit:
        max_profit = profit

    remaining_cash = remaining_cash + sold
    del position[current_symbol.symbol]
    current_symbol.bought = None

In [20]:
automatic_trader(symbol_list)

Buying  288.0 stocks of  AAME At price 2.875 on date 2000-01-28
Buying  61.0 stocks of  AGM At price 20.75 on date 2000-01-28
Buying  10.0 stocks of  AMZN At price 73.38 on date 2000-01-28
Buying  52.0 stocks of  ASRVP At price 23.0 on date 2000-01-28
Buying  14.0 stocks of  AWRE At price 38.0 on date 2000-01-28
Buying  24.0 stocks of  BKH At price 25.0 on date 2000-01-28
Buying  34.0 stocks of  BOSC At price 9.06 on date 2000-01-28
Buying  14.0 stocks of  CACI At price 26.5 on date 2000-01-28
Buying  9.0 stocks of  CAJ At price 42.38 on date 2000-01-28
Buying  14.0 stocks of  CDNS At price 24.63 on date 2000-01-28
Buying  29.0 stocks of  CHH At price 17.38 on date 2000-01-28
Buying  106.0 stocks of  CMU At price 5.875 on date 2000-01-28
Buying  9.0 stocks of  CSPI At price 10.88 on date 2000-01-28
Buying  71.0 stocks of  CYD At price 1.69 on date 2000-01-28
Buying  5.0 stocks of  DGX At price 36.88 on date 2000-01-28
Buying  49.0 stocks of  DNR At price 4.563 on date 2000-01-28
Buying

100%|██████████████████████████████████████████████████████████████████████████████| 3870/3870 [02:30<00:00, 29.48it/s]

Selling  61.0 stocks of AGM  at price 17.38 on date 2000-02-25
Buying  4.0 stocks of  AGYS At price 17.88 on date 2000-02-25
Buying  2.0 stocks of  AIR At price 24.25 on date 2000-02-25
Buying  8.0 stocks of  AIT At price 17.12 on date 2000-02-25
Buying  2.0 stocks of  AVDL At price 8.25 on date 2000-02-25
Buying  2.0 stocks of  BIO At price 29.75 on date 2000-02-25
Buying  2.0 stocks of  BIOL At price 7.0 on date 2000-02-25
Buying  5.0 stocks of  BLK At price 19.25 on date 2000-02-25
Buying  5.0 stocks of  BSTC At price 4.06 on date 2000-02-25
Buying  6.0 stocks of  CAE At price 5.0 on date 2000-02-25
Buying  4.0 stocks of  CARV At price 13.5 on date 2000-02-25
Buying  3.0 stocks of  CBL At price 23.0 on date 2000-02-25
Buying  6.0 stocks of  CCA At price 13.94 on date 2000-02-25
Buying  2.0 stocks of  COLB At price 14.25 on date 2000-02-25
Buying  2.0 stocks of  CPE At price 13.81 on date 2000-02-25
Buying  4.0 stocks of  CVV At price 2.12 on date 2000-02-25
Buying  5.0 stocks of  DW

Buying  2.0 stocks of  MNST At price 5.063 on date 2000-08-11
Selling  2.0 stocks of KEQU  at price 11.88 on date 2000-08-25
Buying  2.0 stocks of  KMM At price 9.0 on date 2000-08-25
Selling  2.0 stocks of VERU  at price 0.52 on date 2000-09-15
Buying  2.0 stocks of  EPM At price 0.66 on date 2000-09-22
Selling  3.0 stocks of DDR  at price 13.25 on date 2000-10-06
Buying  2.0 stocks of  FSI At price 0.81 on date 2000-10-06
Buying  2.0 stocks of  NTX At price 13.38 on date 2000-10-06
Selling  4.0 stocks of AGYS  at price 12.62 on date 2000-10-13
Selling  3.0 stocks of CMCT  at price 9.88 on date 2000-10-20
Selling  9.0 stocks of MCA  at price 13.69 on date 2000-10-20
Buying  2.0 stocks of  MSN At price 2.5 on date 2000-10-20
Buying  3.0 stocks of  MYD At price 13.06 on date 2000-10-20
Selling  2.0 stocks of NVEC  at price 0.31 on date 2000-10-20
Buying  2.0 stocks of  RCS At price 9.563 on date 2000-10-20
Buying  2.0 stocks of  ZTR At price 7.12 on date 2000-10-20
Buying  2.0 stocks of

Buying  2.0 stocks of  APWC At price 2.4 on date 2003-08-29
Buying  2.0 stocks of  SQBG At price 0.65 on date 2003-09-26
Buying  2.0 stocks of  MEET At price 1.8516 on date 2003-10-03
Buying  4.0 stocks of  CRWS At price 0.8789 on date 2003-11-14
Selling  4.0 stocks of CRWS  at price 0.57 on date 2003-11-28
Selling  2.0 stocks of MEET  at price 1.21 on date 2003-12-19
Buying  2.0 stocks of  CRWS At price 0.9 on date 2004-02-06
Selling  2.0 stocks of FIZZ  at price 10.0 on date 2004-04-02
Selling  2.0 stocks of AKR  at price 12.74 on date 2004-04-23
Selling  3.0 stocks of MNR  at price 8.179 on date 2004-05-07
Selling  2.0 stocks of KRC  at price 31.5 on date 2004-05-14
Buying  3.0 stocks of  CMT At price 3.49 on date 2004-05-21
Selling  2.0 stocks of VCV  at price 14.42 on date 2004-05-21
Selling  2.0 stocks of GGP  at price 29.85 on date 2004-06-04
Buying  2.0 stocks of  PFL At price 20.5 on date 2004-06-04
Buying  4.0 stocks of  NYMT At price 9.05 on date 2004-08-27
Buying  2.0 stock

Selling  5.0 stocks of PESI  at price 1.2 on date 2008-11-21
Selling  2.0 stocks of NUV  at price 8.59 on date 2008-12-05
Buying  2.0 stocks of  BPOPM At price 20.5 on date 2008-12-26
Buying  4.0 stocks of  LINU At price 5.3 on date 2009-01-09
Selling  2.0 stocks of PXSV  at price 11.63 on date 2009-01-09
Selling  2.0 stocks of ZTR  at price 3.5 on date 2009-01-09
Selling  2.0 stocks of BPOPM  at price 18.0001 on date 2009-01-30
Buying  2.0 stocks of  MBG At price 27.5 on date 2009-03-20
Buying  2.0 stocks of  AGI At price 7.25 on date 2009-03-27
Buying  2.0 stocks of  HBM At price 5.817 on date 2009-04-09
Buying  2.0 stocks of  NUW At price 15.7 on date 2009-04-17
Buying  2.0 stocks of  NJV At price 15.05 on date 2009-05-22
Selling  3.0 stocks of SES  at price 0.65 on date 2009-05-29
Selling  4.0 stocks of ARE  at price 41.08 on date 2009-06-05
Selling  3.0 stocks of EAD  at price 8.91 on date 2009-06-05
Selling  3.0 stocks of ECF  at price 5.7496 on date 2009-06-05
Selling  3.0 stock

Selling  4.0 stocks of SPHS  at price 0.33 on date 2012-02-10
Selling  7.0 stocks of CO  at price 2.929 on date 2012-03-30
Selling  4.0 stocks of LINU  at price 3.8999 on date 2012-07-20
Buying  3.0 stocks of  LND At price 4.8 on date 2012-10-12
Selling  11.0 stocks of HGSH  at price 0.42 on date 2012-10-19
Selling  2.0 stocks of WMLP  at price 5.77 on date 2012-11-16
Selling  3.0 stocks of LND  at price 4.4 on date 2012-11-23
Buying  3.0 stocks of  SNGX At price 0.77 on date 2012-11-23
Selling  2.0 stocks of NAT  at price 9.13 on date 2012-12-07
Buying  2.0 stocks of  TGLS At price 9.85 on date 2012-12-07
Selling  4.0 stocks of TRNO  at price 16.495 on date 2013-01-18
Buying  2.0 stocks of  LND At price 4.84 on date 2013-01-25
Buying  2.0 stocks of  SPPP At price 10.02 on date 2013-01-25
Selling  2.0 stocks of SPPP  at price 9.06 on date 2013-09-13
Buying  9.0 stocks of  FPAY At price 0.8 on date 2014-03-14
Selling  3.0 stocks of CCCL  at price 1.75 on date 2014-03-21
Selling  9.0 sto

In [21]:
print("Remaining Cash ::" , remaining_cash)

Remaining Cash :: 39.19279999999917


In [22]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)

Maximum Profit is  8478.1653
Minimum Loss is  -2398.0498999999986
Profit 8478.1653
Remaining cash  16488.165300000004
Win Percentage
49.115913555992144
